# CallMeMaybe-virtual-telephony

# Introduccion

 El análisis de datos se ha convertido en una herramienta esencial para optimizar procesos y mejorar la toma de decisiones en servicios de telefonía virtual. Plataformas como CallMeMaybe generan grandes volúmenes de información sobre la actividad de sus operadores, lo que abre la puerta a identificar patrones, detecta áreas de mejora y eleva la calidad del servicio ofrecido a los clientes.

 En este contexto, surge la necesidad de analizar de manera sistemática los registros de llamadas para abordar un desafío concreto: identificar a los operadores menos eficaces dentro de la organización. La ineficacia puede manifestarse a través de indicadores como un alto número de llamadas entrantes perdidas, tiempos de espera prolongados y, en el caso de operadores responsables de llamadas salientes, un bajo volumen de actividad.

 A través de técnicas de análisis exploratorio de datos y la aplicación de métricas clave de desempeño, es posible transformar los datos brutos en información accionable. Este proceso no solo permite detectar a los operadores que requieren apoyo o capacitación adicional, sino que también contribuye a la mejora continua del servicio y la satisfacción de los clientes.

# Objetivo General

Analizar los datos de uso del servicio de telefonía virtual CallMeMaybe para identificar a los operadores menos eficaces y proporcionar información que permita mejorar la gestión y calidad del servicio.



# Descripcion de los datasets: 

El conjunto de datos `telecom_dataset_us.csv` contiene las siguientes columnas:

- `user_id`: ID de la cuenta de cliente
- `date`: fecha en la que se recuperaron las estadísticas
- `direction`: "dirección" de llamada (`out` para saliente, `in` para entrante)
- `internal`: si la llamada fue interna (entre los operadores de un cliente o clienta)
- `operator_id`: identificador del operador
- `is_missed_call`: si fue una llamada perdida
- `calls_count`: número de llamadas
- `call_duration`: duración de la llamada (sin incluir el tiempo de espera)
- `total_call_duration`: duración de la llamada (incluido el tiempo de espera)

El conjunto de datos `telecom_clients_us.csv` tiene las siguientes columnas:

- `user_id`: ID de usuario/a
- `tariff_plan`: tarifa actual de la clientela
- `date_start`: fecha de registro de la clientela

In [411]:
# Importar Librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta


## Carga de los Datasets



In [412]:
# Carga y definición de los Datasets

dataset=pd.read_csv('Data_sets/telecom_dataset_us.csv', sep=",")
clients=pd.read_csv("Data_sets/telecom_clients_us.csv", sep=",")

# Información de los Datasets
print("Dataset de telecomunicaciones:")
dataset.info()

print("\nDataset de clientes:")
clients.info()



Dataset de telecomunicaciones:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              53902 non-null  int64  
 1   date                 53902 non-null  object 
 2   direction            53902 non-null  object 
 3   internal             53785 non-null  object 
 4   operator_id          45730 non-null  float64
 5   is_missed_call       53902 non-null  bool   
 6   calls_count          53902 non-null  int64  
 7   call_duration        53902 non-null  int64  
 8   total_call_duration  53902 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.3+ MB

Dataset de clientes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      732 non-null  

In [413]:
# Muestra de los Datasets
print("\nMuestra del Dataset de telecomunicaciones:")
print(dataset.sample(10))
print(dataset.shape)

print("\nMuestra del Dataset de clientes:")
print(clients.sample(10))
print(clients.shape)



Muestra del Dataset de telecomunicaciones:
       user_id                       date direction internal  operator_id  \
31698   167445  2019-11-05 00:00:00+03:00        in    False     920732.0   
17518   166939  2019-11-24 00:00:00+03:00        in    False          NaN   
53800   168598  2019-11-25 00:00:00+03:00        in    False     958394.0   
2839    166482  2019-09-30 00:00:00+03:00        in    False          NaN   
53237   168412  2019-11-06 00:00:00+03:00       out    False     953458.0   
33467   167497  2019-10-11 00:00:00+03:00        in    False          NaN   
18768   166983  2019-09-13 00:00:00+03:00        in    False     901880.0   
39479   167742  2019-11-22 00:00:00+03:00        in    False     955032.0   
22542   167060  2019-11-22 00:00:00+03:00       out    False     930912.0   
8454    166658  2019-09-13 00:00:00+03:00       out    False     890402.0   

       is_missed_call  calls_count  call_duration  total_call_duration  
31698           False            2 

## DEACUERDO A LA LECTURA DE LOS DATASETS, HACEMOS UN DESGLOSE DEL PROYECTO TENTATIVAMENTE DE LA SIGUIENTE MANERA:.

### POSIBLE ESTRUCTURA DEL PROYECTO, CABE RESALTAR QUE A MEDIDA DEL DESARROLLO DEL PROYECTO, SE PUEDEN HACER AJUSTES A ESTA ESTRUCTURA, CON EL FIN DE MEJORAR LA CALIDAD DEL ANALISIS Y LOS RESULTADOS.

### Análisis Exploratorio de Datos (EDA)
- Cargar y revisar los datasets (`telecom_dataset_us.csv` y `telecom_clients_us.csv`).
- Limpiar y preparar los datos (manejo de valores nulos, tipos de datos, duplicados).
- Analizar la distribución de llamadas por tipo, operador y cliente.
- Visualizar tendencias, patrones y posibles outliers.

### Visualización de Datos
- Crear gráficos para ilustrar la distribución de llamadas entrantes y salientes.
- Comparar la actividad de los operadores mediante gráficos de barras o líneas.
- Visualizar la duración de las llamadas y tiempos de espera.
### Identificación de Operadores y Clientes
- Identificar los operadores únicos y sus respectivos clientes.
- Agrupar los datos por operador y cliente para facilitar el análisis posterior.
- Identificar los tipos de planes de tarifas asociados a cada cliente.
### Análisis de Llamadas Entrantes y Salientes
- Calcular el número total de llamadas entrantes y salientes por operador.
- Analizar la tasa de llamadas perdidas por operador.
- Evaluar el tiempo promedio de espera en llamadas entrantes.
- Calcular la duración promedio de las llamadas atendidas por cada operador.
- Evaluar el volumen total de llamadas gestionadas por cada operador.
### Análisis de Llamadas Internas
- Calcular la tasa de llamadas internas perdidas por operador.
- Evaluar el número de llamadas internas realizadas por cada operador.

### Cálculo de Métricas Clave
- Definir métricas clave de rendimiento (KPIs) para evaluar la eficacia de los operadores.

### Cálculo de KPIs Relevantes
- Definir y calcular los KPIs seleccionados para cada operador.
- Analizar la distribución de los KPIs y comparar entre operadores.

### Identificación de Operadores Ineficaces Basados en KPIs e Bibliografica Actualizada
- Establecer criterios o umbrales para considerar a un operador como ineficaz.
- Listar y visualizar los operadores que cumplen con los criterios de ineficacia.

### Pruebas de Hipótesis Estadísticas teniendo en cuenta el tipo de plan
- Formular hipótesis sobre las diferencias entre operadores eficaces e ineficaces.
- Aplicar pruebas estadísticas para validar las diferencias observadas entre los tipos de planes y posibles similitudes estadisticas o diferencias claves del comportamiento.

### Recomendaciones y Conclusiones
- Generar recomendaciones basadas en los hallazgos.
- Presentar conclusiones y posibles acciones para la mejora del servicio.

---

## KPIs Recomendados

- **Tasa de llamadas entrantes perdidas por operador**  
  Proporción de llamadas entrantes (internas y externas) no atendidas.  
  Fórmula:  
   Tasa de llamadas perdidas = Llamadas entrantes perdidas \ Total de llamadas entrantes

- **Tiempo promedio de espera en llamadas entrantes**  
  Promedio del tiempo de espera antes de que una llamada sea atendida.  
  Fórmula:  
  Tiempo de espera= total_call_duration - call_duration
  

- **Número de llamadas salientes realizadas**  
  Total de llamadas salientes hechas por cada operador.

- **Tasa de llamadas internas perdidas**  
  Proporción de llamadas internas no atendidas por cada operador.

- **Duración promedio de llamada**  
  Promedio de la duración de las llamadas atendidas por cada operador.

- **Volumen total de llamadas gestionadas**  
  Suma total de llamadas (entrantes, salientes, internas) gestionadas por cada operador.

---

## Fuentes Bibliográficas

- [Zendesk: Call Center Metrics and KPIs](https://www.zendesk.com/blog/call-center-metrics/)
- [Talkdesk: The 15 Most Important Call Center Metrics](https://www.talkdesk.com/blog/call-center-metrics/)
- [Freshdesk: Call Center KPIs](https://freshdesk.com/customer-support/call-center-kpis-blog/)
- [ResearchGate: Key Performance Indicators for Call Centers](https://www.researchgate.net/publication/327964964_Key_Performance_Indicators_for_Call_Centers)
- [IBM: Call Center Analytics](https://www.ibm.com/topics/call-center-analytics)

---

# Analisis exploratorio de datos EDA: 

- De acuerdo a la lectura de los datasets, se hace necesario la correcta designacion de los tipos de datos, con el fin de poder establecer las relaciones entre las variables y realizar un analisis mas profundo de los datos.


 - Para fines de analisis, se hace neseario sustituir los valores 'NAN' del Id de operador, ya que estos representandan al rededor de un 15% de los datos, tal porcentaje al ser eliminado puede afectar el analisis de patrones o posibles tendencias.

 - De acuerdo a la natualeza de la informacion y la periodicidad, no sera necesario eliminar duplicados, ya que cada registro representa un dato unico. 
 

In [414]:

# Correcion de tipos de datos en el dataset
dataset['date'] = pd.to_datetime(dataset['date'])
dataset['date']= dataset['date'].dt.date
dataset['date']= pd.to_datetime(dataset['date'], format='%Y-%m-%d')
# Eliminar filas vacias

dataset = dataset.dropna(subset=['internal'])
# Para fines de analisis los valores NAN se sustituiran por 0
dataset['operator_id'] = dataset['operator_id'].fillna(0)

print(dataset.sample(5))

dataset.info()

print(dataset['operator_id'].nunique())

       user_id       date direction internal  operator_id  is_missed_call  \
37934   167645 2019-10-16       out    False     919126.0           False   
12954   166782 2019-10-01       out    False     893804.0            True   
21330   167031 2019-10-14       out    False     910520.0            True   
35502   167532 2019-10-25       out     True     917850.0            True   
20359   167002 2019-10-18       out    False     900492.0            True   

       calls_count  call_duration  total_call_duration  
37934            7            352                  448  
12954           34              0                  877  
21330            5              0                  107  
35502            1              0                    2  
20359            3              0                   16  
<class 'pandas.core.frame.DataFrame'>
Index: 53785 entries, 0 to 53901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               ---------

In [415]:
# Correcion de tipos de datos en el dataset de clientes

clients['date_start'] = pd.to_datetime(clients['date_start'])
clients['date_start']= clients['date_start'].dt.date
clients['date_start']= pd.to_datetime(clients['date_start'], format='%Y-%m-%d')

print(clients.sample(5))
clients.info()




     user_id tariff_plan date_start
353   166604           C 2019-08-12
493   166785           B 2019-08-19
168   168336           C 2019-10-22
695   166530           B 2019-08-07
125   168335           C 2019-10-22
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      732 non-null    int64         
 1   tariff_plan  732 non-null    object        
 2   date_start   732 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 17.3+ KB


## Estadistica descriptiva preliminar para entender la naturaleza de los datos



In [416]:
## Estadísticas descriptivas del Dataset de telecomunicaciones

describe_dataset = dataset.describe()

print("\nDescripción del Dataset de telecomunicaciones:")
print(describe_dataset)


Descripción del Dataset de telecomunicaciones:
             user_id                           date    operator_id  \
count   53785.000000                          53785   53785.000000   
mean   167294.939630  2019-10-23 01:08:38.798921472  778244.735075   
min    166377.000000            2019-08-02 00:00:00       0.000000   
25%    166782.000000            2019-10-04 00:00:00  890416.000000   
50%    167162.000000            2019-10-28 00:00:00  906406.000000   
75%    167819.000000            2019-11-14 00:00:00  930020.000000   
max    168606.000000            2019-11-28 00:00:00  973286.000000   
std       598.998532                            NaN  328641.093195   

        calls_count  call_duration  total_call_duration  
count  53785.000000   53785.000000         53785.000000  
mean      16.482681     868.436162          1159.476174  
min        1.000000       0.000000             0.000000  
25%        1.000000       0.000000            47.000000  
50%        4.000000      38.000